# 基于MindSpore实现Softmax
本实验主要介绍使用MindSpore实现softmax回归，使用随机生成的样本点。

## 1、实验目的

 (1)掌握softmax函数的基本原理。
 
 (2)实现softmax函数，并使用MindSpore应用实现模型定义和交叉熵损失函数，进行分类模型的的训练。

## 2、原理介绍
### softmax原理
Softmax函数是一种数学函数，它将一个数字向量转换为概率分布。它通常用于机器学习和深度学习的分类问题中。Softmax函数用于神经网络的输出层，将输出值转换为概率分布。输出值通过Softmax函数传递，将它们转换为概率。具有最高概率的类被视为预测类。

给定一个样本实例$x_{ij}$，softmax回归模型会先计算出每个类的分数$X_{ij}$，然后对这些分数应用softmax函数，估算出每个类的概率。
计算出每个类的分数，就可以通过softmax函数来评估实例属于类k的概率$\hat p_k$。函数分别计算每个分数的指数，然后对其进行归一化，也就是除以所有指数的总和。<br>
softmax函数：
$$\hat p_k = {softmax}(X_{j})=\frac{exp(X_{j})}{\Sigma_{k}{exp(X_{k})}}.$$
其中k是类数，$X_{j}$是一个向量，包含实例x的每个类的分数 ，j表示第j个分类，$\hat p_k$是实例x属于类k的估计概率，给定当前实例每个类的分数。<br>

### 交叉熵损失函数
交叉熵能够衡量同一个随机变量中的两个不同概率分布的差异程度，在机器学习中就表示为真实概率分布与预测概率分布之间的差异。交叉熵的值越小，模型预测效果就越好。交叉熵经常配合softmax使用，将输出的结果进行处理，使其多个分类的预测值和为1，再通过交叉熵来计算损失。

在二分类的情况下，模型最后预测的结果只有两类，对于每个类别我们的预测得到的概率为p和1-p, 此时表达式为(log的底数是e)：

$$L=\frac{1}{N} \sum_{i} L_{i}=\frac{1}{N} \sum_{i}-\left[y_{i} \cdot \log \left(p_{i}\right)+\left(1-y_{i}\right) \cdot \log \left(1-p_{i}\right)\right]$$

其中，$y_{i}$表示样本i的label，正类为1，负类为0；$p_{i}$表示样本i预测为正类的概率；N为样本总数。



## 3、实验环境
在动手进行实践之前，需要注意以下几点：
* 确保实验环境正确安装，包括安装MindSpore。安装过程：首先登录[MindSpore官网安装页面](https://www.mindspore.cn/install)，根据安装指南下载安装包及查询相关文档。同时，官网环境安装也可以按下表说明找到对应环境搭建文档链接，根据环境搭建手册配置对应的实验环境。
* 推荐使用交互式的计算环境Jupyter Notebook，其交互性强，易于可视化，适合频繁修改的数据分析实验环境。
* 实验也可以在华为云一站式的AI开发平台ModelArts上完成。
* 推荐实验环境：MindSpore版本=MindSpore 2.0；Python环境=3.7


|  硬件平台 |  操作系统  | 软件环境 | 开发环境 | 环境搭建链接 |
| :-----:| :----: | :----: |:----:   |:----:   |
| CPU | Windows-x64 | MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.1节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| GPU CUDA 10.1|Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第二章2.2节和第三章3.1节](./MindSpore环境搭建实验手册.docx)|
| Ascend 910  | Linux-x86_64| MindSpore2.0 Python3.7.5 | JupyterNotebook |[MindSpore环境搭建实验手册第四章](./MindSpore环境搭建实验手册.docx)|

## 4、数据处理

在本次实验中，我们使用的是numpy生成测试数据：
[[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]。

创建一个数据样本y_hat：[[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]]，其中包含2个样本在3个类别的预测概率。

创建它们对应的标签y：[0, 2] 。

## 5、模型构建
实现softmax回归，并使用随机生成的样本点进行测试。

In [2]:
import sys 
#导入与Python解释器和它的环境有关的函数，这里是将搜索路径存放在sys模块的path中。
sys.path.append('..')

In [3]:
import mindspore #导入mindspore框架。
from mindspore import nn #将Model用于创建模型对象，完成网络搭建和编译，并用于训练和评估
from mindspore import ops #导入可用于Cell的构造函数的算子。

给定一个矩阵X，我们可以对所有元素求和。

In [4]:
X = mindspore.Tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], mindspore.float32)
X.sum(0, keepdims=True), X.sum(1, keepdims=True)

(Tensor(shape=[1, 3], dtype=Float32, value=
 [[ 5.00000000e+00,  7.00000000e+00,  9.00000000e+00]]),
 Tensor(shape=[2, 1], dtype=Float32, value=
 [[ 6.00000000e+00],
  [ 1.50000000e+01]]))

### 5.1 实现**softmax**: 
根据下列公式实现softmax，并使用数据测试。

$${softmax}(X)_{ij}=\frac{exp(X_{ij})}{\Sigma_{k}{exp(X_{ik})}}.$$


In [5]:
import mindspore.ops as ops
def softmax(X):
    X_exp = ops.exp(X)
    partition = X_exp.sum(1, keepdims=True)
    return X_exp / partition

我们将每个元素变成一个非负数。此外，依据概率原理，每行总和为1

In [6]:
import numpy as np
X = mindspore.Tensor(np.random.normal(0, 1, (2, 5)), mindspore.float32)
X

Tensor(shape=[2, 5], dtype=Float32, value=
[[-4.29501414e-01,  1.24971855e+00,  8.74681413e-01,  3.25778693e-01,  5.76772332e-01],
 [-1.22778118e+00,  1.21119344e+00, -1.59655130e+00,  4.45182383e-01,  2.74024308e-02]])

In [7]:
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(Tensor(shape=[2, 5], dtype=Float32, value=
 [[ 6.70706779e-02,  3.59590799e-01,  2.47133747e-01,  1.42740324e-01,  1.83464393e-01],
  [ 4.54766974e-02,  5.21221280e-01,  3.14509720e-02,  2.42296860e-01,  1.59554213e-01]]),
 Tensor(shape=[2], dtype=Float32, value= [ 9.99999940e-01,  1.00000000e+00]))

In [8]:
X_prob

Tensor(shape=[2, 5], dtype=Float32, value=
[[ 6.70706779e-02,  3.59590799e-01,  2.47133747e-01,  1.42740324e-01,  1.83464393e-01],
 [ 4.54766974e-02,  5.21221280e-01,  3.14509720e-02,  2.42296860e-01,  1.59554213e-01]])

### 5.2 模型构建
定义模型class Net(nn.Cell)，这里需要用到nn.Cell，因为只有这样才能够使编译器识别编译为计算图，网络输入为样本X。

In [9]:
import mindspore.nn as nn
from mindspore import Tensor, Parameter
from mindspore.common.initializer import initializer, Zero, Normal
class Net(nn.Cell):#定义模型
    def __init__(self, num_inputs, num_outputs):
        super().__init__
        self.W = Parameter(initializer(Normal(0.01, 0), (num_inputs, num_outputs), mindspore.float32))
        self.b = Parameter(initializer(Zero(), num_outputs, mindspore.float32))

    def construct(self, X):
        return softmax(ops.matmul(X.reshape((-1, self.W.shape[0])), self.W) + self.b)

创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率，及它们对应的标签y，使用y作为y_hat中概率的索引。

In [10]:
y = mindspore.Tensor([0, 2], mindspore.int32)#生成标签
y_hat = mindspore.Tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]], mindspore.float32)#生成样本
y_hat[[0, 1], y]

Tensor(shape=[2], dtype=Float32, value= [ 1.00000001e-01,  5.00000000e-01])

实现交叉熵损失函数。

In [11]:
import mindspore.numpy as mnp

def cross_entropy(y_hat, y):#交叉熵损失函数
    return -mnp.log(y_hat[mnp.arange(y_hat.shape[0]), y])

cross_entropy(y_hat, y)

Tensor(shape=[2], dtype=Float32, value= [ 2.30258751e+00,  6.93148613e-01])

### 5.3 模型预测
定义一个计算精确度的函数，输入预测值和真实值，计算模型的精确度。

In [12]:
def accuracy(y_hat, y):  
   #计算预测正确的数量
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.asnumpy() == y.asnumpy()
    return float(cmp.sum())

accuracy(y_hat, y) / len(y)

0.5